Install Needed Packeges

In [ ]:
# !conda install -c conda-forge geocoder
# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c anaconda requests
# !conda install -c anaconda beautifulsoup4
# !conda install -c anaconda lxml

Import Needed Packeges

In [1]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and long

import folium # map rendering library
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

import requests # for making HTTP requests in Python
import pandas as pd # providing high-performance, easy-to-use data structures 
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

import numpy as np # library to handle data in a vectorized manner
from pandas.io.json import json_normalize
from urllib.request import urlopen
import json
from sklearn.cluster import KMeans # import k-means from clustering stage
from bs4 import BeautifulSoup # for pulling data out of HTML and XML files
import geopy.distance
import matplotlib.colors as colors # A module for converting numbers or color arguments to RGB or RGBA

## Get Abu Dhabi City GeoLocation using Nominatim

In [2]:
address = 'Abu Dhabi, Abu Dhabi, United Arab Emirates'
# address = '102 North End Ave, New York, NY'
geolocator = Nominatim(user_agent="ADZss")
location = geolocator.geocode(address)
ADLat = location.latitude
ADLong = location.longitude
print((ADLat,ADLong ))

(24.39647445, 54.5366630864359)


Show Adu Dhabi Map

In [ ]:
folium.Map(location=[ADLat, ADLong],zoom_start=11)

## Get Post Offices in Dhabi City
i will use two methods to retrive post offices in Abu Dhabi<br>
1- Foursquare.com with radius 50 Km of Abu dhabi city location<br>
2- The UAE official Open Data Portal (Bayanat.ae)

#### 1- Foursquare

In [ ]:
CLIENT_ID = 'SETXCY5U2AK53ES13PY4SUB0IK4YQPFGSFXCKOCIUADQJ1U1' # your Foursquare ID
CLIENT_SECRET = 'S3IGI0LFO01E2UHCD5CXLK21K0VAMBWUGMSNVBFT5POUUXTK' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

radius = 50000
LIMIT = 500
search_query = 'Post Office'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ADLat, ADLong, VERSION, search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
# results

In [ ]:
# assign relevant part of JSON to venues
postOffices = results['response']['venues']

# tranform venues into a dataframe
Foursquare_df = json_normalize(postOffices)
# Foursquare_df.head()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories','location.lat','location.lng']  #+ [col for col in Foursquare_df.columns if col.startswith('location')] + ['id']
Foursquare_df_filtered = Foursquare_df.loc[:, filtered_columns]

# filter the category for each row
# Foursquare_df_filtered['categories'] = Foursquare_df_filtered.apply(get_category_type, axis=1)
Foursquare_df_filtered = Foursquare_df_filtered[Foursquare_df_filtered['name'].str.contains("Post Office") | Foursquare_df_filtered['categories'].str.contains("Post Office")]

Foursquare_df_filtered.drop(columns=['categories'], inplace=True)
Foursquare_df_filtered.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)

# Foursquare_df_filtered.head()

#### Bayanat.ae
Bayanat.ae The UAE official Open Data Portal provides enhanced open data systems for public use, 
http://data.bayanat.ae/en_GB/dataset/emirates-post-offices-gis-location

In [3]:
bayanatURL = "http://data.bayanat.ae/api/action/datastore_search?resource_id=44d831dc-2c88-4b37-bc1d-57ab680181df&q=Dhabi"
results = requests.get(bayanatURL).json()

bayanat_df = json_normalize(results["result"]["records"])
# bayanat_df.head()

In [4]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['Branch postoffice name', 'updated Longitude','updated Latitude']
filtered_bayanat_df = bayanat_df.loc[:, filtered_columns]

filtered_bayanat_df.rename(columns={"Branch postoffice name": "name", "updated Longitude": "lng", "updated Latitude": "lat"}, inplace=True)
# filtered_bayanat_df.dtypes

In [5]:
filtered_bayanat_df = filtered_bayanat_df.astype({"name": str, "lng": float,"lat": float})
# filtered_bayanat_df.dtypes

##### Plot Post offices retrived from <font color='blue'>Foursquare</font> and <font color='orange'>bayanat.ae</font> 

In [ ]:
postOffices_map = folium.Map(location=[ADLat, ADLong], zoom_start=11)

# add the post Offices as blue circle markers
for lat, lng, label in zip(Foursquare_df_filtered.lat, Foursquare_df_filtered.lng, Foursquare_df_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(postOffices_map)

for lat, lng, label in zip(filtered_bayanat_df.lat, filtered_bayanat_df.lng, filtered_bayanat_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(postOffices_map)
# display map
postOffices_map

Analysing Plotted Post Office GeoLocations from Plot Post offices retrived from <font color='blue'>Foursquare</font> and <font color='orange'>bayanat.ae</font> <br>

- bayanat.ae have more data
- Foursquare have Doublicates 
 - 2 locations for Butain Post Office
 - 2 locations for Khalidiya Post Office
 - 2 locations for Abu Dhabi Central Post Office
 - 2 locations for Hamdan Post Office
 
 Thus we will carry on using Data from bayanat.ae

### Data Cleanup

#### 1- Drop Post Offices outside Abu Dhabi City

In [6]:
coords_1 = (ADLat, ADLong)
filtered_bayanat_df["distance"]=np.nan

for index, row in filtered_bayanat_df.iterrows():
    coords_2 = (row["lat"], row["lng"])
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    filtered_bayanat_df.loc[index, 'distance'] = distance
    
filtered_bayanat_df.head()

,name,lng,lat,distance
0,Tam Centre - Zayed City Post Office,53.700222,23.674578,116.758158
1,Zayed City Post Office,53.702538,23.659077,117.774249
2,Tam Centre - Ghayathi Post Office,52.809547,23.849718,185.718252
3,Tam Centre - Liwa Post Office,53.798080,23.131104,159.087667
4,Al Nadi Sayahi Post Office,54.379688,24.498306,19.508143


In [7]:
# Drop all rows where distance is greater than 50Km (50000m radius)
filtered_bayanat_df.drop(filtered_bayanat_df[filtered_bayanat_df['distance'] > 50 ].index , inplace=True)
filtered_bayanat_df

,name,lng,lat,distance
4,Al Nadi Sayahi Post Office,54.379688,24.498306,19.508143
5,Bani Yas Post Office,54.643620,24.285358,16.410080
7,Hamdan Post Office,54.366886,24.496719,20.485225
9,Al Falah Post Office,54.379513,24.481054,18.485430
10,Al Musafah Post Office,54.521702,24.377821,2.563666
12,Al Butain Post Office,54.349064,24.459110,20.250433
13,Al Rahba Post Office,54.694187,24.576313,25.529641
14,Khalidiya Post Office,54.340950,24.469082,21.414531
15,Sheikh Khalifa Medical City Post Office,54.364971,24.468798,19.165585
16,Shabiyat Musafah Post Office,54.532993,24.332832,7.059090


In [ ]:
print ("Post Offices from Bayanat.ae:{}\nPost Offices from Foursquare:{}".format(filtered_bayanat_df.shape[0],Foursquare_df_filtered.shape[0])) 

In [8]:
# Drop "distance" Column 
filtered_bayanat_df.drop(columns=['distance'], inplace=True)

Drop "Abu Dhabi International Airport Post Office"

In [9]:
filtered_bayanat_df.drop(filtered_bayanat_df[filtered_bayanat_df['name'] == 'Abu Dhabi International Airport Post Office'].index , inplace=True)
filtered_bayanat_df

,name,lng,lat
4,Al Nadi Sayahi Post Office,54.379688,24.498306
5,Bani Yas Post Office,54.643620,24.285358
7,Hamdan Post Office,54.366886,24.496719
9,Al Falah Post Office,54.379513,24.481054
10,Al Musafah Post Office,54.521702,24.377821
12,Al Butain Post Office,54.349064,24.459110
13,Al Rahba Post Office,54.694187,24.576313
14,Khalidiya Post Office,54.340950,24.469082
15,Sheikh Khalifa Medical City Post Office,54.364971,24.468798
16,Shabiyat Musafah Post Office,54.532993,24.332832


In [ ]:
print ("Row Counts:{}".format(filtered_bayanat_df.shape[0])) 

#### 1- Clean up names, remove "Emirates Post - "

In [ ]:
# filtered_bayanat_df2 = filtered_bayanat_df
# filtered_bayanat_df2['name']

In [ ]:
# filtered_bayanat_df2.loc[:, 'name'].replace(regex=True, to_replace=["Emirates Post - ", "Tam Centre - "], value="",inplace=True)
# filtered_bayanat_df2['name']

## Get Neighbourhoods of Dhabi City
https://en.wikipedia.org/wiki/Abu_Dhabi contains list of Dhabi City Neighbourhoods

we will use BeautifulSoup to scrap the wikipedia page for list of Neighbourhoods

In [10]:
url = "https://en.wikipedia.org/wiki/Abu_Dhabi"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')

table = soup.find('div', attrs={'class':'div-col columns column-width','style':'-moz-column-width: 18em; -webkit-column-width: 18em; column-width: 18em;'})

# print(table)

res = []
ul = table.find('ul')
for li in ul.find_all('li'):
    res.append(li.a.get('title'))

df_ADNeighbourhoods = pd.DataFrame(res, columns=["Neighborhood"])

In [13]:
df_ADNeighbourhoods.head()

,Neighborhood
0,Al Aman (page does not exist)
1,Al Bateen (page does not exist)
2,Al Dhafrah (page does not exist)
3,Al Falah (page does not exist)
4,"Al Karama, United Arab Emirates"


In [12]:
print ("Row Counts:{}".format(df_ADNeighbourhoods.shape[0])) 

Row Counts:43


### Data Cleanup

#### 1- Cleanup Neighborhood Name
remove (page does not exist)" from Neighborhood Name<br>
fix Neighborhood names 

In [14]:
df_ADNeighbourhoods.loc[:, 'Neighborhood'].replace(regex=True, to_replace=[", United Arab Emirates"," \(page does not exist\)"," \(Abu Dhabi\)"," neighbourhood"], value="",inplace=True)

df_ADNeighbourhoods.loc[:, 'Neighborhood'].replace(regex=True, to_replace=["Al Karama"], value="Al Karamah",inplace=True)
df_ADNeighbourhoods.loc[:, 'Neighborhood'].replace(regex=True, to_replace=["Al Maqtaa"], value="Al Maqta",inplace=True)
df_ADNeighbourhoods.loc[:, 'Neighborhood'].replace(regex=True, to_replace=["Al Meena"], value="Al Mina",inplace=True)
df_ADNeighbourhoods.loc[:, 'Neighborhood'].replace(regex=True, to_replace=["Al Moroor"], value="Al Muroor",inplace=True)
df_ADNeighbourhoods.loc[:, 'Neighborhood'].replace(regex=True, to_replace=["Al Khalidyah"], value="al khalidiya",inplace=True)


### Get Neighbourhoods GeoLocations

In [15]:
# Add Columns for latitude and longitude
df_ADNeighbourhoods["lat"]=np.nan
df_ADNeighbourhoods["lng"]=np.nan

In [16]:
# use Nominatim to get Neighbourhoods GeoLocation

geolocator = Nominatim(user_agent="ADZss")
for index, row in df_ADNeighbourhoods.iterrows():
    address = "{},Abu Dhabi, United Arab Emirates".format(row["Neighborhood"])
    try:
        location = geolocator.geocode(address, timeout=10)
        df_ADNeighbourhoods.at[index, 'lat'] = location.latitude
        df_ADNeighbourhoods.at[index, 'lng'] = location.longitude
    except AttributeError:
        print("at index: {}, {} : Problem with data or cannot Geocode.".format(index,row["Neighborhood"]))

    
    


at index: 9, Al Manaseer : Problem with data or cannot Geocode.
at index: 16, Al Muzoon : Problem with data or cannot Geocode.
at index: 18, Al Qubesat : Problem with data or cannot Geocode.
at index: 19, Al Ras Al Akhdar : Problem with data or cannot Geocode.
at index: 22, Al Rehhan : Problem with data or cannot Geocode.
at index: 25, Bani Yas : Problem with data or cannot Geocode.
at index: 26, Al Zaab : Problem with data or cannot Geocode.
at index: 31, Hideriyyat : Problem with data or cannot Geocode.
at index: 33, Marina Village : Problem with data or cannot Geocode.
at index: 40, Qasr El Shatie : Problem with data or cannot Geocode.
at index: 42, Al Samha : Problem with data or cannot Geocode.


In [17]:
#Add Missing Locations (Available on References)
df_ADNeighbourhoods.at[1,['lat','lng']]= {24.4513155,54.328144}

df_ADNeighbourhoods.at[9,['lat','lng']]= {24.453333, 54.363889}
df_ADNeighbourhoods.at[16,['lat','lng']]= {24.4081379,54.4338398}
df_ADNeighbourhoods.at[19,['lat','lng']]= {24.4596244,54.3190142}
df_ADNeighbourhoods.at[25,['lat','lng']]= {24.3063479,54.5995936}
df_ADNeighbourhoods.at[26,['lat','lng']]= {24.4667598,54.356339}
df_ADNeighbourhoods.at[31,['lat','lng']]= {24.417739,54.3576465}
df_ADNeighbourhoods.at[33,['lat','lng']]= {24.4772307,54.3179553}
df_ADNeighbourhoods.at[40,['lat','lng']]= {24.4261292,54.380629}
df_ADNeighbourhoods.at[42,['lat','lng']]= {24.695833,54.676111}

df_ADNeighbourhoods.drop(df_ADNeighbourhoods[df_ADNeighbourhoods['Neighborhood'] == 'Al Qubesat'].index , inplace=True)
df_ADNeighbourhoods.drop(df_ADNeighbourhoods[df_ADNeighbourhoods['Neighborhood'] == 'Al Rehhan'].index , inplace=True)
# Drop 18
# Drop 22
# 25 	Bani Yas, 24.3063479,54.5995936, https://www.google.com/maps/place/Bani+Yas+-+Abu+Dhabi/@24.3063479,54.5995936,13z/data=!3m1!4b1!4m5!3m4!1s0x3e5e371c349ddf2f:0xa49228e3d5f7c257!8m2!3d24.3105884!4d54.643093
# 26 	Al Zaab,24.4667598,54.356339 ,https://www.google.com/maps/place/Al+Zaab+-+Abu+Dhabi/@24.4667598,54.356339,16z/data=!3m1!4b1!4m5!3m4!1s0x3e5e6610cb4267f7:0xbed72ca9fe455d82!8m2!3d24.4653511!4d54.3617472
# 31 	Hideriyyat, 24.4071002,54.3346183, https://www.google.com/maps/place/Al+Hudayriat+Island/@24.4071002,54.3346183,13.5z/data=!4m13!1m7!3m6!1s0x3e5e6f35f2ff0d6f:0x4c12341b13da96f7!2sAl+Hudayriat+Island!3b1!8m2!3d24.4251255!4d54.3412497!3m4!1s0x3e5e6f35f2ff0d6f:0x4c12341b13da96f7!8m2!3d24.4251255!4d54.3412497
# 33 	Al Marina, 24.4772307,54.3179553, https://www.google.com/maps/place/Al+Marina+-+Abu+Dhabi/@24.4772307,54.3179553,14.75z/data=!4m5!3m4!1s0x3e5e65b76b302d3d:0x84129351ceed1e7d!8m2!3d24.477385!4d54.3207573
# 40 	Qasr Al+Shatie, 24.4261292,54.380629, https://www.google.com/maps/place/Qasr+Al+Shatie+-+Abu+Dhabi/@24.4261292,54.380629,16z/data=!3m1!4b1!4m5!3m4!1s0x3e5e68feacd1e88d:0x457d527d98a87b97!8m2!3d24.4257347!4d54.3863512
# 42 	Al Samha,24.695833,54.676111 , https://en.wikipedia.org/wiki/Al_Samha

df_ADNeighbourhoods.head()


,Neighborhood,lat,lng
0,Al Aman,24.431990,54.426550
1,Al Bateen,24.451315,54.328144
2,Al Dhafrah,24.476147,54.369360
3,Al Falah,24.444696,54.728187
4,Al Karamah,24.465300,54.371509


In [18]:
# Add Missing Neighborhood (Al Bahyah, Al Shahama, Al Rahbah)
df_ADNeighbourhoods = df_ADNeighbourhoods.append({'Neighborhood' : 'Al Bahyah' , 'lat' : 24.547706, 'lng' : 54.6628323} , ignore_index=True)
df_ADNeighbourhoods = df_ADNeighbourhoods.append({'Neighborhood' : 'Al Shahama' , 'lat' : 24.524108, 'lng' : 54.6770834} , ignore_index=True)
df_ADNeighbourhoods = df_ADNeighbourhoods.append({'Neighborhood' : 'Al Rahbah' , 'lat' : 24.605807, 'lng' : 54.6944333} , ignore_index=True)

In [ ]:
# address = 'Al Shahamah, Abu Dhabi, United Arab Emirates'
# # address = '102 North End Ave, New York, NY'
# geolocator = Nominatim(user_agent="ADZss")
# location = geolocator.geocode(address)
# print(location.latitude,location.longitude)
# 1 	Al Bateen, 24.4513155,54.328144, https://www.google.com/maps/place/Al+Bateen+-+Abu+Dhabi/@24.4513155,54.3281448,14z/data=!3m1!4b1!4m13!1m7!3m6!1s0x0:0x0!2zMjTCsDEyJzU0LjMiTiA1NcKwMzcnMzQuOCJF!3b1!8m2!3d24.2150834!4d55.6263272!3m4!1s0x3e5e65fca640eac7:0xa0abfabe915f4195!8m2!3d24.4537909!4d54.3523264
# 4 	Al Karamah, ,25.250000, 55.300000 https://en.wikipedia.org/wiki/Al_Karama,_United_Arab_Emirates
# 9 	 Al Manaseer , 24.453333, 54.363889 , http://wikimapia.org/10088815/Al-Manaseer
# 16 	 Al Muzoon , 24.4081379,54.4338398 , https://www.google.com/maps/place/Al+Muzoon+-+Abu+Dhabi/@24.4081379,54.4338398,14.25z/data=!4m5!3m4!1s0x3e5e69e031e23a43:0x37bf7885558ffdeb!8m2!3d24.4104788!4d54.4354121

# 19 	Al Ras Al Akhdar, 24.4596244,54.3190142,https://www.google.com/maps/place/Al+Ras+Al+Akhdar+-+Abu+Dhabi/@24.4596244,54.3190142,15.75z/data=!4m13!1m7!3m6!1s0x3e5e6599c35f9181:0x3afd829c714bf3f1!2sAl+Ras+Al+Akhdar+-+Abu+Dhabi!3b1!8m2!3d24.4601629!4d54.3182858!3m4!1s0x3e5e6599c35f9181:0x3afd829c714bf3f1!8m2!3d24.4601629!4d54.3182858
# 25 	Bani Yas,24.2948788, 54.614033, https://www.google.com/maps/place/Bani+Yas+-+Abu+Dhabi/@24.2948788,54.614033,11.25z/data=!4m5!3m4!1s0x3e5e371c349ddf2f:0xa49228e3d5f7c257!8m2!3d24.3105884!4d54.643093
# 18 	Al Qubesat -- DROP this
# 22 	Al Rehhan -- DROP this
# 25 	Bani Yas, 24.3063479,54.5995936, https://www.google.com/maps/place/Bani+Yas+-+Abu+Dhabi/@24.3063479,54.5995936,13z/data=!3m1!4b1!4m5!3m4!1s0x3e5e371c349ddf2f:0xa49228e3d5f7c257!8m2!3d24.3105884!4d54.643093
# 26 	Al Zaab,24.4667598,54.356339 ,https://www.google.com/maps/place/Al+Zaab+-+Abu+Dhabi/@24.4667598,54.356339,16z/data=!3m1!4b1!4m5!3m4!1s0x3e5e6610cb4267f7:0xbed72ca9fe455d82!8m2!3d24.4653511!4d54.3617472
# 31 	Hideriyyat, 24.417739,54.3576465, https://www.google.com/maps/place/24%C2%B025'03.9%22N+54%C2%B021'34.2%22E/@24.417739,54.3576465,17z/data=!3m1!4b1!4m14!1m7!3m6!1s0x3e5e504f09a79b11:0x581c873c7be01417!2sAl+Bahyah+-+Abu+Dhabi!3b1!8m2!3d24.5480645!4d54.6643482!3m5!1s0x0:0x0!7e2!8m2!3d24.4177392!4d54.3594865
# 33 	Al Marina, 24.4772307,54.3179553, https://www.google.com/maps/place/Al+Marina+-+Abu+Dhabi/@24.4772307,54.3179553,14.75z/data=!4m5!3m4!1s0x3e5e65b76b302d3d:0x84129351ceed1e7d!8m2!3d24.477385!4d54.3207573
# 40 	Qasr Al+Shatie, 24.4261292,54.380629, https://www.google.com/maps/place/Qasr+Al+Shatie+-+Abu+Dhabi/@24.4261292,54.380629,16z/data=!3m1!4b1!4m5!3m4!1s0x3e5e68feacd1e88d:0x457d527d98a87b97!8m2!3d24.4257347!4d54.3863512
# 42 	Al Samha,24.695833,54.676111 , https://en.wikipedia.org/wiki/Al_Samha

# Al+Bahyah, 24.5568273,54.577864, https://www.google.com/maps/place/Al+Bahyah+-+Abu+Dhabi/@24.5568273,54.577864,12z/data=!3m1!4b1!4m13!1m7!3m6!1s0x0:0x0!2zMjTCsDEyJzU0LjMiTiA1NcKwMzcnMzQuOCJF!3b1!8m2!3d24.2150834!4d55.6263272!3m4!1s0x3e5e504f09a79b11:0x581c873c7be01417!8m2!3d24.5480601!4d54.6643639
# Al+Shahama , 24.505878,54.6340008, https://www.google.com/maps/place/Al+Shahama+-+Abu+Dhabi/@24.505878,54.6340008,13z/data=!3m1!4b1!4m13!1m7!3m6!1s0x0:0x0!2zMjTCsDEyJzU0LjMiTiA1NcKwMzcnMzQuOCJF!3b1!8m2!3d24.2150834!4d55.6263272!3m4!1s0x3e5e51cc4d1989bb:0xec3d8ea42d3640fe!8m2!3d24.5240114!4d54.6787834
# Al+Rahbah, 24.5991696,54.6756415, https://www.google.com/maps/place/Al+Rahbah+-+Abu+Dhabi/@24.5991696,54.6756415,14z/data=!3m1!4b1!4m13!1m7!3m6!1s0x0:0x0!2zMjTCsDEyJzU0LjMiTiA1NcKwMzcnMzQuOCJF!3b1!8m2!3d24.2150834!4d55.6263272!3m4!1s0x3e5e519bf23f0341:0xcd5590db2ad175ca!8m2!3d24.6056645!4d54.6966362

##### Plot Abu Dhabi City Neighbourhoods

In [19]:
Neighbourhoods_map = folium.Map(location=[ADLat, ADLong], zoom_start=9) # generate map centred around the Conrad Hotel

mc = MarkerCluster()#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for lat, lng, label in zip(df_ADNeighbourhoods.lat, df_ADNeighbourhoods.lng, df_ADNeighbourhoods.Neighborhood):
    mc.add_child(folium.Marker(location=[lat,lng],popup=label,icon=folium.Icon(color='lightgray', icon='home', prefix='fa')))

heat_data = [[row['lat'],row['lng']] for index, row in df_ADNeighbourhoods.iterrows()]
HeatMap(heat_data).add_to(Neighbourhoods_map)

Neighbourhoods_map.add_child(mc)
# display map
Neighbourhoods_map

## Data Analysis

Calculate total number of post offcies within 2500m radius of a Neighbourhood

In [ ]:
df_ADNeighbourhoods2 = df_ADNeighbourhoods

# add new column for Post Offices Count
df_ADNeighbourhoods2["poCount"]=0

In [ ]:
for index, row in df_ADNeighbourhoods2.iterrows():
    nCoords = (row["lat"], row["lng"])
    for i, r in filtered_bayanat_df2.iterrows():
        poCoords = (r["lat"], r["lng"])
        distance=geopy.distance.geodesic(nCoords,poCoords).m
        if distance <=2500:
            df_ADNeighbourhoods2.at[index,"poCount"] = df_ADNeighbourhoods2.at[index,"poCount"] +1

df_ADNeighbourhoods2.head()

Calculate total number of Neighbourhoods within 2500m radius of a Post Office

In [ ]:
filtered_bayanat_df2 = filtered_bayanat_df

# add new column for Neighbourhoods Count
filtered_bayanat_df2["nbCount"]=0

In [ ]:
# get Neighbourhoods within 2500m raduis of post office
for index, row in filtered_bayanat_df2.iterrows():
    poCoords = (row["lat"], row["lng"])
    for i, r in df_ADNeighbourhoods2.iterrows():
        nCoords = (r["lat"], r["lng"])
        distance=geopy.distance.geodesic(nCoords,poCoords).m
        if distance <=2500:
            filtered_bayanat_df2.at[index,"nbCount"] = filtered_bayanat_df2.at[index,"nbCount"] +1
filtered_bayanat_df2.head()

##### Plot Abu Dhabi City <font color='gray'>Neighbourhoods</font> and <font color='orange'>Post Offices</font>

In [ ]:
Neighbourhoods_map = folium.Map(location=[ADLat, ADLong], zoom_start=11) # generate map centred around the Conrad Hotel

# add the post Offices as orange circle markers
for lat, lng, label,po in zip(df_ADNeighbourhoods.lat, df_ADNeighbourhoods.lng, df_ADNeighbourhoods.Neighborhood,df_ADNeighbourhoods.poCount):
    popup_text = """{}<br>total Post Offices: {}"""
    popup_text = popup_text.format(label,po)
    
    folium.features.Marker([lat, lng],popup=popup_text,icon=folium.Icon(color='lightgray', icon='home', prefix='fa')).add_to(Neighbourhoods_map)

for lat, lng, label,nb in zip(filtered_bayanat_df2.lat, filtered_bayanat_df2.lng, filtered_bayanat_df2.name,filtered_bayanat_df2.nbCount):
    popup_text = """{}<br>total Neighbourhoods: {}"""
    popup_text = popup_text.format(label,nb)
    
    folium.features.Circle(
        [lat, lng],
        radius=2500,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.2
    ).add_to(Neighbourhoods_map)
    folium.features.Marker([lat, lng],popup=popup_text,icon=folium.Icon(color='orange', icon='envelope', prefix='fa')).add_to(Neighbourhoods_map)

# display map
Neighbourhoods_map

In [ ]:
df_ADNeighbourhoods_clust = df_ADNeighbourhoods.drop('Neighborhood', 1)

In [ ]:
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_ADNeighbourhoods_clust)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_ADNeighbourhoods_clust)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
df_ADNeighbourhoods_clustering = df_ADNeighbourhoods
df_ADNeighbourhoods_clustering.insert(1, 'ClusterLabels', kmeans.labels_)

In [ ]:
df_ADNeighbourhoods_clustering.head() # check the last columns!

In [ ]:
# set color scheme for the clusters
rainbow = ['red','green','blue']


In [ ]:
rainbow

In [ ]:
# create map
map_clusters = folium.Map(location=[ADLat, ADLong], zoom_start=11)




map_clusters = folium.Map(location=[ADLat, ADLong], zoom_start=11) # generate map centred around the Conrad Hotel

# add the post Offices as orange circle markers
for lat, lng, label,po,ClusterLabels in zip(df_ADNeighbourhoods.lat, df_ADNeighbourhoods.lng, df_ADNeighbourhoods.Neighborhood,df_ADNeighbourhoods.poCount,df_ADNeighbourhoods.ClusterLabels):
    popup_text = """{}<br>Total Post Offices: {}"""
    popup_text = popup_text.format(label,po)
    
    folium.features.Marker([lat, lng],popup=popup_text,icon=folium.Icon(color=rainbow[ClusterLabels], icon='home', prefix='fa')).add_to(map_clusters)


for lat, lng, label,np in zip(filtered_bayanat_df2.lat, filtered_bayanat_df2.lng, filtered_bayanat_df2.name,filtered_bayanat_df2.nbCount):
    popup_text = """{}<br>Total Neighbourhoods: {}"""
    popup_text = popup_text.format(label,np)
    
    folium.features.Circle(
        [lat, lng],
        radius=2500,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.2
    ).add_to(map_clusters)
    folium.features.Marker([lat, lng],popup=popup_text,icon=folium.Icon(color='orange', icon='envelope', prefix='fa')).add_to(map_clusters)

# display map
map_clusters




In [ ]:
# Examine Cluster 0 , 0 Offices 
df_ADNeighbourhoods.loc[df_ADNeighbourhoods['ClusterLabels'] == 0, df_ADNeighbourhoods.columns[list(range(0, df_ADNeighbourhoods.shape[1]))]]